In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer

# --- 1. GPU/Device Setup ---
# Check if CUDA is available and set the device accordingly.
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# --- Dataset Definition ---
data = {
    "text": [
        "I love this product, it's perfect and exceeded my expectations!", 
        "This application is absolutely terrible and unusable, a complete waste of time.", 
        "It works fine most of the time, no major issues, just average performance.", 
        "I’m so disappointed with the lack of features and constant bugs.", 
        "Absolutely fantastic experience, top-notch support and incredibly quick resolution!", 
        "Horrible service! I waited over an hour for a response and got no help.", 
        "Not bad at all, could be better but it serves its basic purpose well.", 
        "Worst thing ever, I'm canceling my subscription right now, I'm furious.",
        "Great help from support, they were very prompt, efficient, and friendly.", 
        "Okay I guess, nothing special about it, quite neutral actually.",
        "The service was bad and my issue wasn't fixed.",
        "I have no opinion on the matter."
    ],
    # Sentiment labels ranging from -1.0 (very negative) to +1.0 (very positive)
    "label": [0.95, -0.9, 0.3, -0.85, 1.0, -1.0, 0.4, -1.0, 0.85, 0.0, -0.8, 0.0] 
}
df = pd.DataFrame(data)

print(f"Total messages loaded: {len(df)}")
print("\nFirst 5 messages and their labels:")
print(df.head())

Using device: cuda
Total messages loaded: 12

First 5 messages and their labels:
                                                text  label
0  I love this product, it's perfect and exceeded...   0.95
1  This application is absolutely terrible and un...  -0.90
2  It works fine most of the time, no major issue...   0.30
3  I’m so disappointed with the lack of features ...  -0.85
4  Absolutely fantastic experience, top-notch sup...   1.00


In [3]:
# 1. Initialize the vectorizer
# We set max_features to limit the size of the vocabulary (input dimension)
vectorizer = TfidfVectorizer(max_features=500)

# 2. Fit and transform the text data
X = vectorizer.fit_transform(df["text"]).toarray()
y = df["label"].values

# 3. Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Get the resulting input dimension (number of features/words)
INPUT_DIM = X_train.shape[1]

print(f"Total vocabulary size (Input Dimension to NN): {INPUT_DIM}")
print(f"Training data shape (Messages, Features): {X_train.shape}")
print("\nExample of the vectorized data (first message, showing non-zero feature values):")
# Find the non-zero feature indices and values for the first training message
sample_vector = X_train[0]
non_zero_indices = np.nonzero(sample_vector)[0]
feature_names = vectorizer.get_feature_names_out()
sample_features = {feature_names[i]: sample_vector[i] for i in non_zero_indices}
print(sample_features)

Total vocabulary size (Input Dimension to NN): 98
Training data shape (Messages, Features): (9, 98)

Example of the vectorized data (first message, showing non-zero feature values):
{'and': 0.16572233658626395, 'efficient': 0.3203764813039455, 'friendly': 0.3203764813039455, 'from': 0.3203764813039455, 'great': 0.3203764813039455, 'help': 0.27514304368347225, 'prompt': 0.3203764813039455, 'support': 0.27514304368347225, 'they': 0.3203764813039455, 'very': 0.3203764813039455, 'were': 0.3203764813039455}


In [4]:
class SentimentNet(nn.Module):
    """
    Simple Feedforward Network for Sentiment Classification.
    
    The architecture is dynamic based on the hidden_dims_list provided.
    """
    def __init__(self, input_dim, hidden_dims_list):
        super(SentimentNet, self).__init__()
        
        # Define the layer dimensions: Input -> Hidden Layers -> Output (1)
        layer_dims = [input_dim] + hidden_dims_list + [1]
        self.layers = nn.ModuleList()
        
        for i in range(len(layer_dims) - 1):
            # Create a linear layer connecting the current dimension to the next
            self.layers.append(nn.Linear(layer_dims[i], layer_dims[i+1]))
            
    def forward(self, x):
        # Pass through hidden layers with ReLU activation
        for layer in self.layers[:-1]:
            x = F.relu(layer(x))
        
        # Final output layer uses Tanh to push the score between -1 and 1
        return torch.tanh(self.layers[-1](x))

# Define the network dimensions:
# Input_DIM comes from the TF-IDF step (number of features)
# We choose two hidden layers: 128 nodes and 64 nodes
HIDDEN_DIMS = [128, 64] 

# Initialize the model
model = SentimentNet(INPUT_DIM, HIDDEN_DIMS)

print(f"Model initialized with Input Dimension: {INPUT_DIM}")
print(f"Hidden Layers: {HIDDEN_DIMS}")
print("\nModel structure (showing layers and parameters):")
print(model)

Model initialized with Input Dimension: 98
Hidden Layers: [128, 64]

Model structure (showing layers and parameters):
SentimentNet(
  (layers): ModuleList(
    (0): Linear(in_features=98, out_features=128, bias=True)
    (1): Linear(in_features=128, out_features=64, bias=True)
    (2): Linear(in_features=64, out_features=1, bias=True)
  )
)


In [5]:
EPOCHS = 100

# Convert Numpy arrays to PyTorch Tensors
# --- IMPORTANT: Move data to the selected device ---
X_train_tensor = torch.FloatTensor(X_train).to(device)
y_train_tensor = torch.FloatTensor(y_train).unsqueeze(1).to(device) # Needs to be (N, 1) shape

# --- IMPORTANT: Move model to the selected device ---
model.to(device)

# Define Loss and Optimizer
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.01)

# Training Loop
loss_history = []
print(f"Starting training for {EPOCHS} epochs on {device}...")

for epoch in range(EPOCHS):
    model.train() # Set the model to training mode
    optimizer.zero_grad() # Clear previous gradients

    # Forward pass: calculate prediction
    output = model(X_train_tensor)
    
    # Calculate loss
    loss = criterion(output, y_train_tensor)
    
    # Backward pass: calculate gradients
    loss.backward()
    
    # Update weights
    optimizer.step()
    
    loss_history.append(loss.item())
    
    if (epoch + 1) % 20 == 0:
        print(f"Epoch [{epoch+1}/{EPOCHS}], Loss: {loss.item():.4f}")

print("\nTraining complete.")
print(f"Final training loss: {loss_history[-1]:.4f}")

Starting training for 100 epochs on cuda...
Epoch [20/100], Loss: 0.0110
Epoch [40/100], Loss: 0.0044
Epoch [60/100], Loss: 0.0036
Epoch [80/100], Loss: 0.0036
Epoch [100/100], Loss: 0.0036

Training complete.
Final training loss: 0.0036


In [6]:
# Set model to evaluation mode (disables dropout, batch norm, etc., if present)
model.eval()

# --- Example 1: Strong Positive ---
new_text_pos = "The support agent was fantastic and solved my issue instantly! I'm delighted."

# Preprocessing: Must use the same vectorizer
new_vec_pos = vectorizer.transform([new_text_pos]).toarray()
# --- IMPORTANT: Move tensor to the selected device for inference ---
new_tensor_pos = torch.FloatTensor(new_vec_pos).to(device) 

with torch.no_grad():
    prediction_pos = model(new_tensor_pos).item()

# --- Example 2: Strong Negative ---
new_text_neg = "This is broken and infuriating. My account is locked and nobody is helping."

new_vec_neg = vectorizer.transform([new_text_neg]).toarray()
# --- IMPORTANT: Move tensor to the selected device for inference ---
new_tensor_neg = torch.FloatTensor(new_vec_neg).to(device)

with torch.no_grad():
    prediction_neg = model(new_tensor_neg).item()

# --- Example 3: Neutral ---
new_text_neu = "I have no preference about the color of the application."

new_vec_neu = vectorizer.transform([new_text_neu]).toarray()
# --- IMPORTANT: Move tensor to the selected device for inference ---
new_tensor_neu = torch.FloatTensor(new_vec_neu).to(device)

with torch.no_grad():
    prediction_neu = model(new_tensor_neu).item()

print("--- Sentiment Predictions (Score range: -1.0 to 1.0) ---")

print(f"\nMessage: '{new_text_pos}'")
print(f"Predicted Sentiment: {prediction_pos:.4f} (Expected: Positive)")

print(f"\nMessage: '{new_text_neg}'")
print(f"Predicted Sentiment: {prediction_neg:.4f} (Expected: Negative)")

print(f"\nMessage: '{new_text_neu}'")
print(f"Predicted Sentiment: {prediction_neu:.4f} (Expected: Neutral/Zero)")

--- Sentiment Predictions (Score range: -1.0 to 1.0) ---

Message: 'The support agent was fantastic and solved my issue instantly! I'm delighted.'
Predicted Sentiment: 0.4009 (Expected: Positive)

Message: 'This is broken and infuriating. My account is locked and nobody is helping.'
Predicted Sentiment: -0.9999 (Expected: Negative)

Message: 'I have no preference about the color of the application.'
Predicted Sentiment: -0.9930 (Expected: Neutral/Zero)
